In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_gp
import ariel_load
import ariel_numerics
import time
import numpy as np
import cupy as cp
import multiprocess
from tqdm import tqdm
import scipy
import copy
import matplotlib.pyplot as plt

local


In [2]:
train_data = kgs.load_all_train_data()
train_data = train_data
kgs.debugging_mode = 1
model = ariel_gp.PredictionModel()
model.run_in_parallel = False
model.model_options.n_iter=0
model.model_options.use_training_labels = True
kgs.sanity_checks_active = True
model.train(train_data)

In [3]:
loaded_res = kgs.dill_load(kgs.temp_dir + '/prep.pickle')
model.infer(train_data[0:1]);

In [4]:
for d,r in (zip(train_data,loaded_res)):
    mm= copy.deepcopy(model.results['model_mean'])
    mm.set_parameters(r[0])
    mm.m['signal'].m['main'].m['transit'].depth_model.offset = -d.diagnostics['training_spectrum']
    obs_transit = copy.deepcopy(model.results['obs'])
    
    stellar_labels = mm.m['signal'].m['main'].m['spectrum'].get_prediction(obs_transit)
    transit_labels = mm.m['signal'].m['main'].m['transit'].get_prediction(obs_transit)
    obs_transit.labels = stellar_labels*transit_labels
    #obs_transit.labels = model.results['model_mean'].m['signal'].m['main'].m['transit'].get_prediction(obs_transit)
    #obs_transit.labels = mm.m['signal'].m['main'].get_prediction(obs_transit)
    #obs_transit.labels = mm.m['signal'].get_prediction(obs_transit)
    #print(mm.m['signal'].m['main'].m['transit'].get_parameters())
    d.diagnostics['transit'] = obs_transit

In [5]:
S,components,weights = kgs.dill_load(kgs.temp_dir + '/explore_bad_pixels_pca.pickle')

In [ ]:
def f(d):
    kgs.debugging_mode = 1
    #model = kgs.dill_load(kgs.temp_dir + '/parallel.pickle')
    #d = model.infer([d])[0]
    d.load_to_step(3, ariel_load.default_loaders())
    #obs_transit = d.diagnostics['transit']
    coeffs = []
    for i_wavelength in range(283):
        if i_wavelength==0:
            # times_obs = np.unique(obs_transit.df['time'][~obs_transit.df['is_AIRS']])
            # transit_obs = obs_transit.export_matrix(False)[:,0]
            # times_data = d.transits[0].data[0].times.get()/3600   
            # transit_data = cp.array(scipy.interpolate.interp1d(times_obs, transit_obs, kind='cubic', fill_value="extrapolate")(times_data))
            # times_data = cp.array(times_data)     
            # data = d.transits[0].data[0].data
            # data = np.reshape(data, (data.shape[0],-1))
            # print(len(times_obs),len(times_data))
            #transit_data = cp.array(obs_transit.export_matrix(False)[:,0])
            data = d.transits[0].data[0].data
            data = np.reshape(data, (data.shape[0],-1))
        else:
            data = d.transits[0].data[1].data[:,:,i_wavelength-1]
            
    
        #noise_est = cp.ones(data.shape[1])
        #for ii in range(data.shape[1]):
        #   noise_est[ii] = ariel_numerics.estimate_noise_cp(data[:,ii])
        noise_est = ariel_numerics.estimate_noise_cp(data)
        noise_est[cp.isnan(noise_est)]=cp.nanmean(noise_est)
        noise_est[noise_est<1] = 1

        # Fit the PCA components to the data
        #design_matrix = cp.stack([c[0,:] for c in components[i_wavelength]]+[cp.ones(data.shape[1])]).T
        design_matrix = cp.stack([c[0,:] for c in components[i_wavelength]]).T
        coeffs.append(ariel_numerics.lstsq_nanrows_normal_eq_with_pinv_sigma(data.T, design_matrix, sigma=noise_est, return_A_pinv_w=False)[0])
        
    d.load_to_step(0, ariel_load.default_loaders())
    return coeffs
 

res = [f(d) for d in tqdm(train_data)]
kgs.dill_save(kgs.temp_dir+'/explore_base_shape_from_pca.pickle', res)
res = kgs.dill_load(kgs.temp_dir+'/explore_base_shape_from_pca.pickle')

 21%|████████████████▋                                                               | 230/1099 [08:55<36:26,  2.52s/it]

In [ ]:
xx=[]
for i_wavelength in tqdm(range(283)):
    if i_wavelength==0:
        transit_combined = np.concatenate([d.diagnostics['transit'].export_matrix(False)[:,0] for d in train_data])
    else:
        transit_combined = np.concatenate([d.diagnostics['transit'].export_matrix(True)[:,i_wavelength-1] for d in train_data])
    coeffs = cp.concatenate([r[i_wavelength] for r in res],1)
    residual = []
    for ii in range(1,coeffs.shape[0]+1):
        X = coeffs[:ii,:]
        coeffs2, _, _, _ = cp.linalg.lstsq(X.T, cp.array(transit_combined), rcond=None)  # coefs shape: (2, M)
        pred = coeffs2@X
        residual.append(kgs.rms(pred-cp.array(transit_combined)).get())
    xx.append(residual)

In [ ]:
plt.figure()
plt.semilogy(xx[0])
plt.figure()
plt.semilogy(xx[1])
plt.figure()
plt.imshow(np.log(np.array(xx[1:]).T), aspect='auto', interpolation='none')
plt.colorbar()

In [ ]:
xx[0][-1],xx[1][-1],xx[-1][-1]